In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
from imageio import imread
import numpy as np
import os
from os.path import join
import train_interactions
import pandas as pd
from interactions_results import toarray, tostruct
import train_interactions
import warnings

In [ ]:
DATA_DIR = '/home/matej/prace/ferda/data/interactions'
EXPERIMENT_DIR = '/home/matej/prace/ferda/experiments/171201_1700_batch/'

NAMES = 'ant1_x, ant1_y, ant1_major, ant1_minor, ant1_angle, ' \
        'ant2_x, ant2_y, ant2_major, ant2_minor, ant2_angle'
FORMATS = 10 * 'f,'

pd.set_option('precision', 2)

In [ ]:
with h5py.File(join(DATA_DIR, 'results_inter_test.h5'), 'r') as hf:
    y_test = hf['data'][:]
    y_test_df = pd.DataFrame(tostruct(y_test))

with h5py.File(join(EXPERIMENT_DIR, '0.172413793103', 'predictions.h5'), 'r') as hf:
    pred = hf['data'][:]
    pred_df = pd.DataFrame(tostruct(pred))

In [ ]:
loss_alpha_batch = pd.read_csv(join(EXPERIMENT_DIR, 'results.csv'))

In [ ]:
loss_alpha_batch.iloc[[5]]

In [ ]:
loss_alpha_batch['xy MAE'].iloc[-1] = np.nan
xy = loss_alpha_batch['xy MAE']
loss_alpha_batch['xy MAE norm'] = (xy - xy.mean()) / xy.std()
loss_alpha_batch['angle MAE'].iloc[[0, -1]] = np.nan
angle = loss_alpha_batch['angle MAE']
loss_alpha_batch['angle MAE norm'] = (angle - angle.mean()) / angle.std()
loss_alpha_batch['sum norm'] = loss_alpha_batch['angle MAE norm'] + loss_alpha_batch['xy MAE norm']
loss_alpha_batch['alpha sum norm'] = loss_alpha_batch['angle MAE norm'] * (1 - loss_alpha_batch['loss_alpha']) + \
                                     loss_alpha_batch['xy MAE norm'] * loss_alpha_batch['loss_alpha']

In [ ]:
ax = loss_alpha_batch.plot('loss_alpha', ['xy MAE norm', 'angle MAE norm', 'alpha sum norm'], 
                           title='normalized mean absolute errors')
ax.legend(['xy', 'angle', 'alpha weighted sum'])
ax.set_xlabel('alpha')

In [ ]:
ax = loss_alpha_batch.plot('loss_alpha', ['xy MAE', 'angle MAE'], title='mean absolute errors')
ax.legend(['xy', 'angle'])
ax.set_ylabel('deg, px')
ax.set_xlabel('alpha')

# Predictions

In [ ]:
pred_df.describe()

In [ ]:
_ = plt.hist2d(pred_df['ant1_x'], pred_df['ant1_y'], bins=40, range=((0, 199), (0, 199)))

In [ ]:
_ = plt.hist2d(pred_df['ant2_x'], pred_df['ant2_y'], bins=40, range=((0, 199), (0, 199)))

In [ ]:
pred_df['ant1_angle'].hist()

In [ ]:
(pred_df['ant1_angle'] % 180).hist()

In [ ]:
pred_df['ant2_angle'].hist()

In [ ]:
(pred_df['ant2_angle'] % 180).hist()

# Prediction Errors

In [ ]:
# xy1_error = np.linalg.norm(toarray(y_test[['ant1_x', 'ant1_y']]) - toarray(pred[['ant1_x', 'ant1_y']]), 2, axis=1)
# xy2_error = np.linalg.norm(toarray(y_test[['ant2_x', 'ant2_y']]) - toarray(pred[['ant2_x', 'ant2_y']]), 2, axis=1)
# angle1_error, angle2_error = train_interactions.angle_absolute_error(toarray(y_test), toarray(pred), np)
# xy1_error_df = pd.DataFrame(xy1_error)
# xy2_error_df = pd.DataFrame(xy2_error)
# angle1_error_df = pd.DataFrame(angle1_error)
# angle2_error_df = pd.DataFrame(angle2_error)

# xy, angle, indices = train_interactions.match_pred_to_gt(pred, y_test, np)
# xy_errors = (xy[indices[:, 0], indices[:, 1]])
# angle_errors = (angle[indices[:, 0], indices[:, 1]])

# swap = indices[:, 0] == 1
# pred[swap, :5], pred[swap, 5:] = pred[swap, 5:], pred[swap, :5]

In [ ]:
xy, angle, indices = train_interactions.match_pred_to_gt(pred, y_test, np)
xy_errors = (xy[indices[:, 0], indices[:, 1]])
angle_errors = (angle[indices[:, 0], indices[:, 1]])

# swap = indices[:, 0] == 1
# pred_swapped = pred.copy()
# pred_swapped[swap, :5], pred_swapped[swap, 5:] = pred_swapped[swap, 5:], pred_swapped[swap, :5]

In [ ]:
df = pd.DataFrame.from_items([('xy (px)', [xy_errors.mean()]), 
                              ('angle (deg)', angle_errors.mean()),])
df.style.set_caption('MAE')
df

In [ ]:
_  = plt.hist(xy_errors)

In [ ]:
_ = plt.hist(angle_errors)

# Model

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

model = train_interactions.model()
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
# SVG(model_to_dot(model.get_layer('model_1'), show_shapes=True).create(prog='dot', format='svg'))